combine 0:no expression and 4:random into a single 0:ignore class

at inference time, we don't care about the difference

the class imbalance might also be useful - as this will be our do nothing default - and we only want to take action when we're sure a command is being requested

see what happens if we don't use nose landmarks for training - try with/without normalizing for nose

different 
- Try bigger batch size - even if it means we need more epochs
- model size/depth
- Exaggerate 0 imbalance - how far do we have to go to not got any 0s misclassified (on test set)combine 0:no expression and 4:random into a single 0:ignore class

at inference time, we don't care about the difference

the class imbalance might also be useful - as this will be our do nothing default - and we only want to take action when we're sure a command is being requested

see what happens if we don't use nose landmarks for training - try with/without normalizing for nose

In [ ]:
# run this, then restart the runtime
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

'[' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

ModuleNotFoundError: No module named 'fastai'

In [ ]:
dataset_id = '20211014143136'
path = Path('/content/drive/MyDrive/Colab Notebooks/datasets')
df = pd.read_csv(path/f'data_{dataset_id}.csv')

In [ ]:
df

In [ ]:
# make all x and y values relative to tip of nose
for i in range(468):
    df[f'{i}x']=df[f'{i}x']-df['1x']
    df[f'{i}y']=df[f'{i}y']-df['1y']

In [ ]:
df

In [ ]:
# calculate stats for normalization
stats = {}
for i in range(468):
    for coord in ['x','y','z']: 
        col=f'{i}{coord}'
        stats[col]={}
        stats[col]['mean']=df[col].mean()
        stats[col]['std']=df[col].std()

In [ ]:
# save these stats so we can use them at inference time
import json
with open(path/f'stats_data_{dataset_id}.json', 'w') as f:
    json.dump(stats, f)

In [ ]:
# normalize
for i in range(468):
    if i == 1: continue # it's all zeros
    for coord in ['x','y','z']:
        col=f'{i}{coord}'
        df[col]=df[col]-stats[col]['mean']
        df[col]=df[col]/stats[col]['std']

In [ ]:
df

In [ ]:
df['expression_id'].value_counts()

In [ ]:
# combine 0:no expression and 4:random into a single 0:ignore class
df.loc[df['expression_id']==4, 'expression_id']=0

In [ ]:
face_point_ids = [5, 2, 218, 438] # up,down,left,right - no point having 1 tip_of_nose as it's always 0
mouth_landmarks = [0, 11, 12, 13, 14, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 57, 61, 62, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 106, 146, 164, 165, 167, 178, 179, 180, 181, 182, 183, 184, 185, 186, 191, 204, 267, 268, 269, 270, 271, 272, 273, 287, 291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 335, 375, 391, 393, 402, 403, 404, 405, 406, 407, 408, 409, 410, 415, 424]

In [ ]:
cont_names = []
for i in face_point_ids+mouth_landmarks:
# for i in range(468):
    cont_names.append(f'{i}x')
    cont_names.append(f'{i}y')
cat_names = []
procs = []

In [ ]:
len(face_point_ids+mouth_landmarks)

In [ ]:
# splits = RandomSplitter(valid_pct=0.2)(range_of(df))
# to = TabularPandas(df, procs=procs,
#                    cat_names = cat_names,
#                    cont_names = cont_names,
#                    y_names='expression_id',
#                    splits=splits)
# dls = to.dataloaders(bs=64)

In [ ]:
# dls.show_batch()

In [ ]:
dls = TabularDataLoaders.from_df(df, path, procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names="expression_id", y_block=CategoryBlock,
                                 valid_idx=list(range(0, len(df), 10)), bs=64)
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.loss_func

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(25, 8e-4, wd=1e-2) # 25, 8e-4 -> 97.4 + wd=1e-2 -> 97.7

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.model

In [ ]:
class_id_to_label = {
        0: "No expression",
        1: "oo",
        2: "ee",
        3: "ah",
        4: "Random Talking"
    }

In [ ]:
confusion_matrix = np.zeros([5,5], dtype=int) # TODO: don't hard code
output = np_model(df[cont_names].to_numpy())
preds = np.argmax(output, axis=1)
targets = df['expression_id'].to_numpy()
for p,t in zip(preds, targets):
    confusion_matrix[t][p]+=1

In [ ]:
confusion_matrix

In [ ]:
# confusion_matrix = np.array([[0,2,1],[1,2,0],[2,1,0]])
fig, ax = plt.subplots(figsize=(9,9))
ax.matshow(confusion_matrix, cmap=plt.cm.Blues, alpha=0.8)
# ax.xaxis.set_ticks_position('bottom') # must be after matshow
for i in range(confusion_matrix.shape[0]):
    for j in range(confusion_matrix.shape[1]):
        ax.text(x=j, y=i,s=confusion_matrix[i, j], va='center', ha='center', size='xx-large')
plt.title('Confusion Matrix', fontsize=14)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
tick_marks = np.arange(5)
# plt.xticks(tick_marks, self.data.y.classes, rotation=90)
plt.xticks(tick_marks, class_id_to_label.values(), rotation=90)
plt.yticks(tick_marks, class_id_to_label.values(), rotation=0)
plt.show()

In [ ]:
import pickle
class NpModelHelper:
    def __init__(self, path, model_id):
        self.model_id, self.path = model_id, path/f'model_{model_id}.pkl'
    def set_state(self, model):
        self.state_dict, state_dict = {}, learn.model.state_dict()
        for k in state_dict:
            self.state_dict[k] = state_dict[k].detach().cpu().numpy()
        return self
    def save(self):
        # can't set allow_pickle=False with np.savez https://github.com/numpy/numpy/issues/13983
        # so we might as well pickle and keep the dict order
        # TODO: don't need the dict order any more - go back to npz
        with open(self.path, 'wb') as f:
            pickle.dump(self.state_dict, f)
        return self
    def load(self):
        with open(self.path, 'rb') as f:
            self.state_dict = pickle.load(f)
        return self
    def get_state_dict(self, name_prefix):
        if name_prefix is None or name_prefix == '': 
            return self.state_dict
        return {k[len(name_prefix)+1:]:self.state_dict[k] 
                for k in self.state_dict 
                if k.startswith(name_prefix)}

In [ ]:
np_model_helper = NpModelHelper(path, dataset_id)
np_model_helper.set_state(learn.model).save()

In [ ]:
np_model_helper.get_state('bn_cont').keys()

In [ ]:
np_model_helper = NpModelHelper(path, dataset_id).load()

In [ ]:
np_model_helper = NpModelHelper(path, dataset_id).load()
np_model = NpModel([
                    NpBatchNorm1d(**np_model_helper.get_state_dict('bn_cont')),
                    NpLinear(**np_model_helper.get_state_dict('layers.0.0')),
                    NpReLU(),
                    NpBatchNorm1d(**np_model_helper.get_state_dict('layers.0.2')),
                    NpLinear(**np_model_helper.get_state_dict('layers.1.0')),
                    NpReLU(),
                    NpBatchNorm1d(**np_model_helper.get_state_dict('layers.1.2')),
                    NpLinear(**np_model_helper.get_state_dict('layers.2.0'))
                   ])

In [ ]:
def module_params_to_np(module, name):
    t = getattr(module, name)
    if t is not None:
        t = t.detach().cpu().numpy()
    return t

def model_params_to_np(m):
    # currently just for fastai TabularModel
    result = {}
    def _add(name_prefix, module, names):
        for name in names:
            result[f'{name_prefix}{name}'] = module_params_to_np(module, name)
    _add('bn_cont#', m.bn_cont, ['weight','bias','running_mean','running_var']) # BatchNorm1d
    _add('layers#0#0#', m.layers[0][0], ['weight','bias']) # Linear
    _add('layers#0#2#', m.layers[0][2], ['weight','bias','running_mean','running_var'])
    _add('layers#1#0#', m.layers[1][0], ['weight','bias'])
    _add('layers#1#2#', m.layers[1][2], ['weight','bias','running_mean','running_var'])
    _add('layers#2#0#', m.layers[2][0], ['weight','bias'])
    return result

In [ ]:
 np_params = model_params_to_np(learn.model)
 np.savez(path/'model_20211014143136.npz', **np_params) # TODO: save without pickle

In [ ]:
class NpBatchNorm1d:
    # https://github.com/pytorch/pytorch/blob/420b37f3c67950ed93cd8aa7a12e673fcfc5567b/aten/src/ATen/native/Normalization.cpp#L61-L126
    def __init__(self, weight, bias, running_mean, running_var, num_batches_tracked=None):
        self.weight, self.bias = weight, bias
        self.running_mean, self.running_std = running_mean, np.sqrt(running_var + 1e-5)
    def __call__(self, x):
        x = x - self.running_mean
        x = x / self.running_std
        x = x * self.weight
        x = x + self.bias
        return x

In [ ]:
class NpLinear:
    def __init__(self, weight, bias=None):
        self.weight, self.bias = weight.T, bias
    def __call__(self, x):
        x = x @ self.weight
        if self.bias is not None:
            x = x + self.bias
        return x

In [ ]:
class NpReLU:
    def __call__(self, x):
        return np.maximum(x, 0)

In [ ]:
class NpModel:
    def __init__(self, modules):
        self.modules = modules
    def __call__(self, x):
        for module in self.modules:
            x = module(x)
        return x

In [ ]:
npz_file = np.load(path/'model_20211014143136.npz', allow_pickle=True)
def _fix_none(arr):
    return None if arr.shape == () else arr
def np_params_from_npz(name_prefix, names):
    return [_fix_none(npz_file[f'{name_prefix}{name}']) for name in names]
np_model = NpModel([
                    NpBatchNorm1d(*np_params_from_npz('bn_cont#', ['weight','bias','running_mean','running_var'])),
                    NpLinear(*np_params_from_npz('layers#0#0#', ['weight','bias'])),
                    NpReLU(),
                    NpBatchNorm1d(*np_params_from_npz('layers#0#2#', ['weight','bias','running_mean','running_var'])),
                    NpLinear(*np_params_from_npz('layers#1#0#', ['weight','bias'])),
                    NpReLU(),
                    NpBatchNorm1d(*np_params_from_npz('layers#1#2#', ['weight','bias','running_mean','running_var'])),
                    NpLinear(*np_params_from_npz('layers#2#0#', ['weight','bias']))
                   ])

In [ ]:
output = np_model(df[cont_names].to_numpy())
preds = np.argmax(output, axis=1)
targets = df['expression_id'].to_numpy()
for i, (p,t) in enumerate(zip(preds,targets)):
    if p!=t:
        print(i,p,t)

In [ ]:
learn.predict(df.iloc[9924])

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
for i in [9922, 9923, 9924, 9925]:
    _df = df.iloc[[i]]
    output = np_model(_df[cont_names].to_numpy())
    preds = np.argmax(output, axis=1)
    targets = _df['expression_id'].to_numpy()
    fastai_preds = learn.predict(df.iloc[i])
    print(i,softmax(output),preds,targets,'\n\n')
    print(fastai_preds,'\n\n')

# This &darr; is the old way of saving model params

In [ ]:
class NpBatchNorm1d:
    # https://github.com/pytorch/pytorch/blob/420b37f3c67950ed93cd8aa7a12e673fcfc5567b/aten/src/ATen/native/Normalization.cpp#L61-L126
    def __init__(self, weight, bias, running_mean, running_var):
        self.weight, self.bias = weight, bias
        self.running_mean, self.running_var = running_mean, running_var
        self.running_std = np.sqrt(running_var + 1e-5)
    def __call__(self, x):
        x = x - self.running_mean
        x = x / self.running_std
        x = x * self.weight
        x = x + self.bias
        return x
    def params(self):
        return dict(weight=self.weight, bias=self.bias, 
                    running_mean=self.running_mean, running_var=self.running_var)

In [ ]:
class NpLinear:
    def __init__(self, weight, bias):
        self.weight, self.bias = weight.T, bias
    def __call__(self, x):
        x = x @ self.weight
        if self.bias is not None:
            x = x + self.bias
        return x
    def params(self):
        return dict(weight=self.weight, bias=self.bias)

In [ ]:
class NpReLU:
    def __call__(self, x):
        return np.maximum(x, 0)
    def params(self):
        return {}

In [ ]:
class NpModel:
    def __init__(self, modules):
        self.modules = modules
    def __call__(self, x):
        for module in self.modules:
            x = module(x)
        return x
    def params(self):
        result = {}
        for i, module in enumerate(self.modules):
            name_prefix = f'{i}_{module.__class__.__name__}_'
            module_params = module.params()
            for k in module_params:
                result[f'{name_prefix}{k}'] = module_params[k]
        return result

In [ ]:
def get_weights_as_np(module, names):
    result = []
    for name in names:
        t = getattr(module, name)
        if t is not None:
            t = t.detach().cpu().numpy()
        result.append(t)
    return result

In [ ]:
m = learn.model
np_model = NpModel([
                    NpBatchNorm1d(*get_weights_as_np(m.bn_cont, ['weight','bias','running_mean','running_var'])),
                    NpLinear(*get_weights_as_np(m.layers[0][0], ['weight','bias'])),
                    NpReLU(),
                    NpBatchNorm1d(*get_weights_as_np(m.layers[0][2], ['weight','bias','running_mean','running_var'])),
                    NpLinear(*get_weights_as_np(m.layers[1][0], ['weight','bias'])),
                    NpReLU(),
                    NpBatchNorm1d(*get_weights_as_np(m.layers[1][2], ['weight','bias','running_mean','running_var'])),
                    NpLinear(*get_weights_as_np(m.layers[2][0], ['weight','bias']))
                   ])

In [ ]:
np.savez(path/'model_20211013102908.npz', **np_model.params())

In [ ]:
output = np_model(df[cont_names].to_numpy())

In [ ]:
preds = np.argmax(output, axis=1)
targets = df['expression_id'].to_numpy()
for i, (p,t) in enumerate(zip(preds,targets)):
    if p!=t:
        print(i,p,t)

In [ ]:
df['expression_id'].to_numpy()

In [ ]:

m

In [ ]:
F.relu( l(x.cuda()))

In [ ]:
np_l = NpLinear(*get_weights_as_np(l, ['weight','bias']))

In [ ]:
NpReLU()( np_l(x.numpy()))

In [ ]:
l = m.layers[0][0]
l

In [ ]:
l.bias

In [ ]:
x = torch.ones([1,212])

In [ ]:
m.bn_cont(x.cuda())

In [ ]:
np_bn = NpBatchNorm1d(*[getattr(m.bn_cont, n).detach().cpu().numpy() for n in ['weight','bias','running_mean','running_var']])

In [ ]:
np_bn(x) # np_bn(x.numpy())

In [ ]:
np.sqrt( np_bn.running_var)

In [ ]:
m = learn.model

In [ ]:
m.bn_cont.eps

In [ ]:
m.bn_cont.running_mean.shape, m.bn_cont.running_var.shape

In [ ]:
m.bn_cont.

In [ ]:
for p in m.bn_cont.parameters():
    print(p.shape)

In [ ]:
m = nn.Sequential(
    nn.Linear(len(face_point_ids+mouth_landmarks)*2, 200, bias=False),
    nn.ReLU(),
    nn.Linear(200, 100, False),
    nn.ReLU(),
    nn.Linear(100, 4, True)
)

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.m=nn.Sequential(
            nn.Linear(len(face_point_ids+mouth_landmarks)*2, 200, bias=False),
            nn.ReLU(),
            nn.Linear(200, 100, False),
            nn.ReLU(),
            nn.Linear(100, 4, True)
        )
                
    def forward(self, x_cat, x_cont):
        x=self.m(x_cont)
        return x

In [ ]:
learn=TabularLearner(dls, SimpleModel(),
                     loss_func=CrossEntropyLossFlat(),
                     wd=1e-3, wd_bn_bias=True)
learn.lr_find()

In [ ]:
learn.fit_one_cycle(15, 1e-3)